# 二、自然语言处理之模型应用----阅读理解

HuggingFace有一个巨大的模型库，其中一些是已经非常成熟的经典模型，这些模型即使不进行任何训练也能直接得出比较好的预测结果，也就是常说的Zero Shot Learning。

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1) 下载模型

In [2]:
# 下载模型
#!HF_ENDPOINT=https://hf-mirror.com hf download distilbert/distilbert-base-cased-distilled-squad --local-dir ../models/distilbert/distilbert-base-cased-distilled-squad

### 2) 使用pipeline加载模型

使用管道工具时，调用者需要做的只是告诉管道工具要进行的任务类型，管道工具会自动分配合适的模型，直接给出预测结果，如果这个预测结果对于调用者已经可以满足需求，则不再需要再训练。

管道工具的API非常简洁，隐藏了大量复杂的底层代码，即使是非专业人员也能轻松使用。

In [3]:
# 在线加载模型
from transformers import pipeline
question_answerer=pipeline(task="question-answering", 
                           model="../models/distilbert/distilbert-base-cased-distilled-squad",
                           device=device) # 目前建议用这个代码

Device set to use cuda


### 3) 查看模型的配置信息

In [4]:
print(question_answerer.model.config)

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "transformers_version": "4.57.1",
  "vocab_size": 28996
}



In [5]:
# 定义一个上下文
context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on
that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/PyTorch/question-
answering/run_squad.py script.
"""

# 定义两个问题
question1 = "What is extractive question answering?"
question2 = "What is a good example of a question answering dataset?"

### 5) 使用模型预测¶

In [6]:
result = question_answerer(question=question1, context=context)
print(result)

result = question_answerer(question=question2, context=context)
print(result)

# 这里是以字节为单位

{'score': 0.6149131655693054, 'start': 34, 'end': 95, 'answer': 'the task of extracting an answer from a text\ngiven a question'}
{'score': 0.517393706504663, 'start': 147, 'end': 160, 'answer': 'SQuAD dataset'}


### 6) 使用from_pretrained加载本地模型，使用底层一点的函数实现

参考这个页面 https://huggingface.co/transformers/v2.11.0/model_doc/distilbert.html

和这个页面 https://huggingface.co/distilbert-base-cased-distilled-squad

In [7]:
# 加载本地模型，使用底层一点的函数
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
tokenizer = DistilBertTokenizer.from_pretrained('../models/distilbert/distilbert-base-cased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('../models/distilbert/distilbert-base-cased-distilled-squad')

# 第一个问题
inputs = tokenizer(question1, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
answer_start_index = int(torch.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(torch.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
result = tokenizer.decode(predict_answer_tokens)
print(answer_start_index, answer_end_index, result) # 这里是以token为单位

# 第二个问题
inputs = tokenizer(question2, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
answer_start_index = int(torch.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(torch.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
result = tokenizer.decode(predict_answer_tokens)
print(answer_start_index, answer_end_index, result) # 这里是以token为单位

15 27 the task of extracting an answer from a text given a question
44 49 SQuAD dataset
